In [14]:
import pandas as pd 
import spotipy 
import json
import time
import re

sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
cid ="73ae7d6ba1134544beb303a0bfade356" 
secret = "4e7f26c2bc44499ea80ab14258a37fb3" 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False 

In [2]:
# this works ;)
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

def show_album_tracks(album):
    tracks = []
    results = sp.album_tracks(album['id'])
    tracks.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    for track in tracks:
        #print('Track Name: ', track['name'])
        tn = track['name']
        track_name.append(tn)
        #print(track['uri'])
        tu = track['uri']
        track_uri.append(tu)
        artist_rep.append(artist['name']) #add artist name to df
        #print()

def show_artist_albums(id):
    albums = []
    results = sp.artist_albums(artist['id'], album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    unique = set()  # skip duplicate albums
    for album in albums:
        name = album['name'].lower()
        
        if not name in unique:  
            #print('Album Name:',name)
            unique.add(name)
            album_name.append(name)  # make list of albums
            show_album_tracks(album)

def show_artist(artist):
    an = artist['name']
    print(artist['name'])
    artist_sp_name.append(an)    # make list of artist names



In [3]:
artists80 = ['Bon Jovi','Fleetwood Mac','Rod Stewart','Michael Jackson','Rush','Elton John','Bruce Springsteen',
             'Aerosmith','Paul McCartney','Depeche Mode','Cyndi Lauper', 'David Bowie','Queen' 
              ]

In [4]:
artist_sp_name = []
album_name = []
track_name = []
track_uri = []
artist_rep = []
for star in artists80:
    try:
        artist = get_artist(star)
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue        
    try:
        show_artist(artist)
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue
    try:
        show_artist_albums(artist)
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue
    

Bon Jovi
Fleetwood Mac
Rod Stewart
Michael Jackson
Rush
Elton John
Bruce Springsteen
Aerosmith
Paul McCartney
Depeche Mode
Cyndi Lauper
David Bowie
Queen


In [5]:
tracks = pd.DataFrame()
tracks['uri']= track_uri
tracks['name'] = track_name
tracks['artist_rep'] = artist_rep

In [6]:
tracks.head()

,uri,name,artist_rep
0,spotify:track:1UHIxScBvmK3riBnKHPY4c,This House Is Not For Sale - Live From The Lon...,Bon Jovi
1,spotify:track:2i7vjApy25wdbZWHAvAaGM,Living With The Ghost - Live From The London P...,Bon Jovi
2,spotify:track:444zbUam6qDxyXt0xp5cXY,Knockout - Live From The London Palladium,Bon Jovi
3,spotify:track:6c9LwjdQj5A248kGw8ZXkm,Labor Of Love - Live From The London Palladium,Bon Jovi
4,spotify:track:4unKLI7od1EkMrofONtwgM,Born Again Tomorrow - Live From The London Pal...,Bon Jovi


In [7]:
features_df = pd.DataFrame(columns = ['acousticness','analysis_url','danceability','duration_ms','energy','id','instrumentalness','key',
                  'liveness','loudness','mode','speechiness','tempo','time_signature','track_href','type','uri','valence'])
for tid in tracks['uri']:
    try:
        features = sp.audio_features(tid) 
        features_df = features_df.append(features)
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue

Connection refused by the server..
Was a nice sleep, now let me continue...
Connection refused by the server..
Was a nice sleep, now let me continue...
Connection refused by the server..
Was a nice sleep, now let me continue...
Connection refused by the server..
Was a nice sleep, now let me continue...
Connection refused by the server..
Was a nice sleep, now let me continue...
Connection refused by the server..
Was a nice sleep, now let me continue...


In [8]:
features_df.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.006010,https://api.spotify.com/v1/audio-analysis/1UHI...,0.512,396173,0.786,1UHIxScBvmK3riBnKHPY4c,0.000023,7,0.95,-7.438,1,0.061,122.049,4,https://api.spotify.com/v1/tracks/1UHIxScBvmK3...,audio_features,spotify:track:1UHIxScBvmK3riBnKHPY4c,0.36
0,0.000306,https://api.spotify.com/v1/audio-analysis/2i7v...,0.405,305293,0.870,2i7vjApy25wdbZWHAvAaGM,0.000070,6,0.977,-6.487,1,0.0612,128.795,4,https://api.spotify.com/v1/tracks/2i7vjApy25wd...,audio_features,spotify:track:2i7vjApy25wdbZWHAvAaGM,0.475
0,0.006010,https://api.spotify.com/v1/audio-analysis/444z...,0.525,218533,0.886,444zbUam6qDxyXt0xp5cXY,0.000026,10,0.702,-5.440,0,0.0665,138.124,4,https://api.spotify.com/v1/tracks/444zbUam6qDx...,audio_features,spotify:track:444zbUam6qDxyXt0xp5cXY,0.433
0,0.178000,https://api.spotify.com/v1/audio-analysis/6c9L...,0.535,351333,0.436,6c9LwjdQj5A248kGw8ZXkm,0.011600,8,0.962,-10.919,1,0.0305,103.299,4,https://api.spotify.com/v1/tracks/6c9LwjdQj5A2...,audio_features,spotify:track:6c9LwjdQj5A248kGw8ZXkm,0.303
0,0.000628,https://api.spotify.com/v1/audio-analysis/4unK...,0.466,247760,0.936,4unKLI7od1EkMrofONtwgM,0.000007,2,0.749,-5.491,0,0.0479,126.882,4,https://api.spotify.com/v1/tracks/4unKLI7od1Ek...,audio_features,spotify:track:4unKLI7od1EkMrofONtwgM,0.491


In [9]:
track_features =  pd.merge(tracks,features_df, on='uri', how='inner')

In [10]:
track_features.head()

,uri,name,artist_rep,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,valence
0,spotify:track:1UHIxScBvmK3riBnKHPY4c,This House Is Not For Sale - Live From The Lon...,Bon Jovi,0.006010,https://api.spotify.com/v1/audio-analysis/1UHI...,0.512,396173,0.786,1UHIxScBvmK3riBnKHPY4c,0.000023,7,0.95,-7.438,1,0.061,122.049,4,https://api.spotify.com/v1/tracks/1UHIxScBvmK3...,audio_features,0.36
1,spotify:track:2i7vjApy25wdbZWHAvAaGM,Living With The Ghost - Live From The London P...,Bon Jovi,0.000306,https://api.spotify.com/v1/audio-analysis/2i7v...,0.405,305293,0.870,2i7vjApy25wdbZWHAvAaGM,0.000070,6,0.977,-6.487,1,0.0612,128.795,4,https://api.spotify.com/v1/tracks/2i7vjApy25wd...,audio_features,0.475
2,spotify:track:444zbUam6qDxyXt0xp5cXY,Knockout - Live From The London Palladium,Bon Jovi,0.006010,https://api.spotify.com/v1/audio-analysis/444z...,0.525,218533,0.886,444zbUam6qDxyXt0xp5cXY,0.000026,10,0.702,-5.440,0,0.0665,138.124,4,https://api.spotify.com/v1/tracks/444zbUam6qDx...,audio_features,0.433
3,spotify:track:6c9LwjdQj5A248kGw8ZXkm,Labor Of Love - Live From The London Palladium,Bon Jovi,0.178000,https://api.spotify.com/v1/audio-analysis/6c9L...,0.535,351333,0.436,6c9LwjdQj5A248kGw8ZXkm,0.011600,8,0.962,-10.919,1,0.0305,103.299,4,https://api.spotify.com/v1/tracks/6c9LwjdQj5A2...,audio_features,0.303
4,spotify:track:4unKLI7od1EkMrofONtwgM,Born Again Tomorrow - Live From The London Pal...,Bon Jovi,0.000628,https://api.spotify.com/v1/audio-analysis/4unK...,0.466,247760,0.936,4unKLI7od1EkMrofONtwgM,0.000007,2,0.749,-5.491,0,0.0479,126.882,4,https://api.spotify.com/v1/tracks/4unKLI7od1Ek...,audio_features,0.491


In [11]:
track_features.shape

(8092, 20)

In [12]:
# this function cleans the end of a track (space hyphen space)
def clean_track(track_features):
    new_track = re.split(r"\b - \b",track_features['name'], 1)[0]
    return new_track

In [15]:
# apply function and create new column
track_features['name_clean']  = track_features.apply(clean_track, axis=1)

In [16]:
track_features.head()

,uri,name,artist_rep,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,valence,name_clean
0,spotify:track:1UHIxScBvmK3riBnKHPY4c,This House Is Not For Sale - Live From The Lon...,Bon Jovi,0.006010,https://api.spotify.com/v1/audio-analysis/1UHI...,0.512,396173,0.786,1UHIxScBvmK3riBnKHPY4c,0.000023,...,0.95,-7.438,1,0.061,122.049,4,https://api.spotify.com/v1/tracks/1UHIxScBvmK3...,audio_features,0.36,This House Is Not For Sale
1,spotify:track:2i7vjApy25wdbZWHAvAaGM,Living With The Ghost - Live From The London P...,Bon Jovi,0.000306,https://api.spotify.com/v1/audio-analysis/2i7v...,0.405,305293,0.870,2i7vjApy25wdbZWHAvAaGM,0.000070,...,0.977,-6.487,1,0.0612,128.795,4,https://api.spotify.com/v1/tracks/2i7vjApy25wd...,audio_features,0.475,Living With The Ghost
2,spotify:track:444zbUam6qDxyXt0xp5cXY,Knockout - Live From The London Palladium,Bon Jovi,0.006010,https://api.spotify.com/v1/audio-analysis/444z...,0.525,218533,0.886,444zbUam6qDxyXt0xp5cXY,0.000026,...,0.702,-5.440,0,0.0665,138.124,4,https://api.spotify.com/v1/tracks/444zbUam6qDx...,audio_features,0.433,Knockout
3,spotify:track:6c9LwjdQj5A248kGw8ZXkm,Labor Of Love - Live From The London Palladium,Bon Jovi,0.178000,https://api.spotify.com/v1/audio-analysis/6c9L...,0.535,351333,0.436,6c9LwjdQj5A248kGw8ZXkm,0.011600,...,0.962,-10.919,1,0.0305,103.299,4,https://api.spotify.com/v1/tracks/6c9LwjdQj5A2...,audio_features,0.303,Labor Of Love
4,spotify:track:4unKLI7od1EkMrofONtwgM,Born Again Tomorrow - Live From The London Pal...,Bon Jovi,0.000628,https://api.spotify.com/v1/audio-analysis/4unK...,0.466,247760,0.936,4unKLI7od1EkMrofONtwgM,0.000007,...,0.749,-5.491,0,0.0479,126.882,4,https://api.spotify.com/v1/tracks/4unKLI7od1Ek...,audio_features,0.491,Born Again Tomorrow


In [17]:
# remove dupes 
track_features = track_features.drop_duplicates(subset=['name_clean', 'artist_rep'])

In [18]:
track_features.shape

(4087, 21)

In [19]:
# download list:
filename = 'spotify.csv'
track_features.to_csv(filename, index=False, encoding='utf-8')

http://web.stanford.edu/~jangjj/spotify-audio-features/index.html

Definitions:
    Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
    
    Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
    
    The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.
    
    Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
    
    Acoustic is a confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
    
    Instrumental predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
    
    Liveness detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
    
    Valence is a measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
    
    Tempo - the overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.